## The purpose of the Capstone 2 project is to determine whether or not the lesion in an image is malignant or not 

In [6]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from skimage.io import imread, imshow, concatenate_images
from skimage.transform import resize
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os
from numpy import array, newaxis, expand_dims

Using TensorFlow backend.
/home/seo/environments/my_env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/seo/environments/my_env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/seo/environments/my_env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/seo/environments/my_env/lib/p

In [7]:
%run functions.py

In [8]:
path="/home/seo/ISIC_DATA/"
process_path='/home/seo/jupyter/Capstone2'
filelist=os.listdir(path)
processfiles=os.listdir(process_path)
meta=[file for file in filelist if not file.endswith(".jpeg") and not file.endswith(".png") and file.startswith("ISIC")]
pics=[file for file in filelist if (file.endswith(".jpeg") or file.endswith(".png")) and file.startswith("ISIC")]

In [9]:
import pickle
model=pickle.load(open('model_fitted.pickle',"rb"))

W0929 21:26:04.795032 139959537661760 deprecation_wrapper.py:119] From /home/seo/environments/my_env/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0929 21:26:04.845596 139959537661760 deprecation_wrapper.py:119] From /home/seo/environments/my_env/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4185: The name tf.truncated_normal is deprecated. Please use tf.random.truncated_normal instead.

W0929 21:26:04.912568 139959537661760 deprecation_wrapper.py:119] From /home/seo/environments/my_env/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:245: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0929 21:26:04.914076 139959537661760 deprecation_wrapper.py:119] From /home/seo/environments/my_env/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated

In [10]:
# Set some parameters
im_width = 128
im_height = 128
img_channels=3
images = np.zeros((len(pics),im_height, im_width, 1), dtype=np.float32)

test_img=call_image(test)
test2=resize(test_img, (128, 128, 1), mode = 'constant', 
                  preserve_range = True)
imshow(array_resize(test_img).squeeze())
imshow(test2.squeeze())

for every picture
call image
denoise
resize
apply mask to get only lesion
have label as dictionary key
save every 1999 pictures

In [ ]:
i=0
resized_dict=dict()
orig_dict=dict()
for pic_name in pics:
    name=path+pic_name
    label=pic_name.split('.')[0]
    tag1="resized"
    img=call_image(name)

    denoised_img=denoise(img)
    resized=array_resize(denoised_img)
    resized_dict[label]=resized
    if len(resized_dict)%1999==0:
        i+=1
        save_name1='{}_{}.p'.format(tag1,i)
        pickle.dump(resized_dict, open(save_name1,"wb"))
        del resized_dict
        resized_dict=dict()
resized_files=[file for file in processfiles if file.split('_')[0]==tag1]

In [7]:
tag1="resized"
resized_files=[file for file in processfiles if file.split('_')[0]==tag1]
resized_files

['resized_1.p',
 'resized_3.p',
 'resized_8.p',
 'resized_6.p',
 'resized_5.p',
 'resized_11.p',
 'resized_9.p',
 'resized_4.p',
 'resized_2.p',
 'resized_7.p',
 'resized_10.p']

In [9]:
open_file=[pickle.load(open(file,'rb')) for file in resized_files]

In [14]:
type(open_file)

list

As we will be working a lot with pickled data, a function was created to 

In [11]:
def load_all_pickles(tag_name, path ,delimiter):
    processfiles=os.listdir(path)
    agg_dict=dict()
    tagged_files=[file for file in processfiles if file.split(delimiter)[0]==tag_name]
    for file in tagged_files:
        open_file=pickle.load(open(file,'rb'))
        agg_dict.update(open_file)
    return agg_dict

In [12]:
resized=load_all_pickles('resized',process_path,'_')

In [ ]:
i=0
pred_dict=dict()
for key in resized.keys():
    tag2="pred"
    img=resized_dict_open[key]
    _,pred = prediction(model,img)
    mask=mask_w_predicted(pred)
    pred_dict[key]=mask
    if len(pred_dict)%1999==0:
        i+=1
        save_name2='{}_{}.p'.format(tag2,i)
        pickle.dump(pred_dict, open(save_name2,"wb"))
        del pred_dict
        pred_dict=dict()
pred_files=[file for file in processfiles if file.split('_')[0]==tag2]

In [17]:
pred=load_all_pickles('pred',process_path,'_')

In [18]:
len(pred)

21989

In [1]:
import pickle
#make a big dictionary with all the images
master_dict=dict()
for i in range(2,8):
    name='segmented_{}.p'.format(i)
    temp=pickle.load(open(name,'rb'))
    master_dict.update(temp)

In [2]:
master_dict

{'ISIC_0027855_segmented': array([[[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        ...,
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]]], dtype=uint8),
 'ISIC_0031595_segmented': array([[[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
    

In [ ]:
i=0
cont_dict=dict()
seg_dict=dict()
for key in pred_dict_open.keys():
    tag3="segmen"
    tag4="contour"
    mask=pred_dict_open[key]
    try:
        cnts,h=cv2.findContours(mask,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE )
        contour=cnts[0]
    except:
        continue
    try:
        M = cv2.moments(contour)
        (a,b),(MA,ma),angle = cv2.fitEllipse(contour)
    except:
        continue
    cont_dict[key]=[cnts[0],angle,int(a),int(b)]    
    try:
        orig_img=call_image(path+key+'.jpeg')
        resized_orig=resize(orig_img, (128, 128, 1), mode = 'constant', 
                            preserve_range = True)
        dst = cv2.bitwise_and(resized_orig, resized_orig, mask=mask)
    except:
        continue
    seg_dict[key]=dst
    if len(cont_dict)%1999==0:
        i+=1
        save_name3='{}_{}.p'.format(tag3,i)
        save_name4='{}_{}.p'.format(tag4,i)
        pickle.dump(cont_dict, open(save_name3,"wb"))
        pickle.dump(seg_dict, open(save_name4,"wb"))
        del cont_dict
        del seg_dict
        cont_dict=dict()
        seg_dict=dict()


In [56]:
seg_dict=pickle.load(open('resized_1.p','rb'))
seg_dict2=pickle.load(open('resized_0.p','rb'))

In [58]:
seg_dict

{'ISIC_0002698': array([[[[0.38681534],
          [0.44230843],
          [0.45288753],
          ...,
          [0.5695839 ],
          [0.55342978],
          [0.46828888]],
 
         [[0.44647823],
          [0.52046943],
          [0.53006583],
          ...,
          [0.6775702 ],
          [0.6537336 ],
          [0.54490288]],
 
         [[0.44347121],
          [0.52881947],
          [0.52910788],
          ...,
          [0.69787279],
          [0.66742317],
          [0.54670867]],
 
         ...,
 
         [[0.44875947],
          [0.54131615],
          [0.55251849],
          ...,
          [0.58491584],
          [0.56629621],
          [0.48774912]],
 
         [[0.43591598],
          [0.52886342],
          [0.54485385],
          ...,
          [0.57038462],
          [0.56924715],
          [0.48656744]],
 
         [[0.372731  ],
          [0.44192068],
          [0.46070099],
          ...,
          [0.47974943],
          [0.49070973],
          [0.40940155]]